In [25]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt','r') # input
# data and labels are loaded from file 
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('labels.txt','r') # ground reality
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [26]:

print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)



labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...


In [27]:

from collections import Counter
import numpy as np

In [28]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

for i in range(0,len(reviews)):
    l=None
    l=reviews[i].split(' ')
    if labels[i] == 'NEGATIVE' or  labels[i] == 'negative' :
        for word in l:
            negative_counts[word]+=1
            total_counts[word] +=1
    else:
        for word in l:
            positive_counts[word]+=1
            total_counts[word] +=1
    
        
            
            
        
        
    

In [29]:
pos_neg_ratios = Counter()
for word in total_counts:
    pos_neg_ratios[word]= (positive_counts[word]+1)/float (negative_counts[word]+1)
    
    

In [30]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 1.060805434849134
Pos-to-neg ratio for 'amazing' = 4.026615969581749
Pos-to-neg ratio for 'terrible' = 0.1781609195402299


In [31]:
for word in pos_neg_ratios:
    pos_neg_ratios[word]=np.log(pos_neg_ratios[word])

In [32]:
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 0.05902846378582202
Pos-to-neg ratio for 'amazing' = 1.3929263134236418
Pos-to-neg ratio for 'terrible' = -1.7250680947293828


pos_neg_diff = Counter()
for word in total_counts:
    pos_neg_diff[word]= 1 + positive_counts[word] - negative_counts[word]
    
for word in pos_neg_diff:
    pos_neg_diff[word]=np.log(pos_neg_diff[word])



# INPUT/OUTPUT

In [33]:
vocab = None

In [34]:
vocab=set(total_counts)

In [35]:
vocab_size = len(vocab)
print(vocab_size)

74074


In [36]:
layer_0=np.array(np.zeros((1,len(vocab))))

In [37]:
layer_0.shape

(1, 74074)

In [38]:
# Create a dictionary of words in the vocabulary mapped to index positions
# (to be used in layer_0)
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
    
# display the map of words to indices


In [39]:
def update_input_layer(review):
    l=None
    global layer_0
    layer_0*=0
    l=review.split(' ')
    for word in l:
        i=word2index[word]
        layer_0[0][i]+=pos_neg_ratios[word]
        

In [40]:
def get_target_for_label(label):
    if label == 'POSITIVE':
        return 1
    else:
        return 0
   

In [41]:
# Encapsulate our neural network in a class

import time
import sys
import numpy as np


class SentimentNetwork:
    def __init__(self, reviews, labels, hidden_nodes = 16, learning_rate = 0.1):
        np.random.seed(1)
        self.pre_process_data(reviews, labels)
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)

    def pre_process_data(self, reviews, labels):
        
        review_vocab = set()
        # TODO: populate review_vocab with all of the words in the given reviews
        #       Remember to split reviews into individual words 
        #       using "split(' ')" instead of "split()".
        
        for review in reviews:
            for word in review.split(' '):
                review_vocab.add(word)
        
        # Convert the vocabulary set to a list so we can access words via indices
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        # TODO: populate label_vocab with all of the words in the given labels.
        #       There is no need to split the labels because each one is a single word.
        for label in labels:
            label_vocab.add(label)
        
        # Convert the label vocabulary set to a list so we can access labels via indices
        self.label_vocab = list(label_vocab)
        
        # Store the sizes of the review and label vocabularies.
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        # Create a dictionary of words in the vocabulary mapped to index positions
        self.word2index = {}
        # TODO: populate self.word2index with indices for all the words in self.review_vocab
        #       like you saw earlier in the notebook
        
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
        # Create a dictionary of labels mapped to index positions
        self.label2index = {}
        # TODO: do the same thing you did for self.word2index and self.review_vocab, 
        #       but for self.label2index and self.label_vocab instead
        
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Store the number of nodes in input, hidden, and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Store the learning rate
        self.learning_rate = learning_rate

        # Initialize weights
        
        # TODO: initialize self.weights_0_1 as a matrix of zeros. These are the weights between
        #       the input layer and the hidden layer.
        self.weights_0_1 = np.zeros([self.input_nodes, self.hidden_nodes])
        
        # TODO: initialize self.weights_1_2 as a matrix of random values. 
        #       These are the weights between the hidden layer and the output layer.
        self.weights_1_2 = np.random.normal(0, 1, [self.hidden_nodes, self.output_nodes])
        
        # TODO: Create the input layer, a two-dimensional matrix with shape 
        #       1 x input_nodes, with all values initialized to zero
        self.layer_0 = np.zeros((1,input_nodes))
    
        
    def update_input_layer(self,review):
        # TODO: You can copy most of the code you wrote for update_input_layer 
        #       earlier in this notebook. 
        #
        #       However, MAKE SURE YOU CHANGE ALL VARIABLES TO REFERENCE
        #       THE VERSIONS STORED IN THIS OBJECT, NOT THE GLOBAL OBJECTS.
        #       For example, replace "layer_0 *= 0" with "self.layer_0 *= 0"
        self.layer_0 *= 0
        for word in review.split(' '):
            if word in self.word2index.keys():
                self.layer_0[0][self.word2index[word]] = pos_neg_ratios[word]
            
    def get_target_for_label(self,label):
        # TODO: Copy the code you wrote for get_target_for_label 
        #       earlier in this notebook. 
        if label == 'POSITIVE': return 1
        else: return 0
        
    def sigmoid(self,x):
        # TODO: Return the result of calculating the sigmoid activation function
        #       shown in the lectures
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_output_2_derivative(self,output):
        # TODO: Return the derivative of the sigmoid activation function, 
        #       where "output" is the original output from the sigmoid fucntion 
        return output * (1 - output)

    def train(self, training_reviews, training_labels):
        
        # make sure out we have a matching number of reviews and labels
        assert(len(training_reviews) == len(training_labels))
        
        # Keep track of correct predictions to display accuracy during training 
        correct_so_far = 0
        
        # Remember when we started for printing time statistics
        start = time.time()

        # loop through all the given reviews and run a forward and backward pass,
        # updating weights for every item
        for i in range(len(training_reviews)):
            
            # TODO: Get the next review and its correct label
            review = training_reviews[i]
            label = training_labels[i]
            
            self.update_input_layer(review)
            
            # TODO: Implement the forward pass through the network. 
            #       That means use the given review to update the input layer, 
            #       then calculate values for the hidden layer,
            #       and finally calculate the output layer.
            #       Do not use an activation function for the hidden layer,
            #       but use the sigmoid activation function for the output layer.
            self.layer_1 = self.layer_0.dot(self.weights_0_1)
            self.layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))
            
            # TODO: Implement the back propagation pass here. 
            #       That means calculate the error for the forward pass's prediction
            err = self.layer_2 - self.get_target_for_label(label)
            layer_2_delta = err * self.sigmoid_output_2_derivative(self.layer_2)
            
            self.weights_1_2 -= self.learning_rate * self.layer_1.T.dot(layer_2_delta)
            #self.weights_0_1 -= layer_2_delta(self.weights_1_2.dot(self.layer_0).T)
            self.weights_0_1 -= self.learning_rate * self.layer_0.T.dot(layer_2_delta.dot(self.weights_1_2.T))
            
            #       and update the weights in the network according to their
            #       contributions toward the error, as calculated via the
            #       gradient descent and back propagation algorithms you 
            #       learned in class.
            
            # TODO: Keep track of correct predictions. To determine if the prediction was
            #       correct, check that the absolute value of the output error 
            #       is less than 0.5. If so, add one to the correct_so_far count.
            
            if(self.layer_2 >= 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            elif(self.layer_2 < 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the training process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) \
                             + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
            if(i % 2500 == 0):
                print("")
    
    def test(self, testing_reviews, testing_labels):
        """
        Attempts to predict the labels for the given testing_reviews,
        and uses the test_labels to calculate the accuracy of those predictions.
        """
        
        # keep track of how many correct predictions we make
        correct = 0

        # we'll time how many predictions per second we make
        start = time.time()

        # Loop through each of the given reviews and call run to predict
        # its label. 
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            # For debug purposes, print out our prediction accuracy and speed 
            # throughout the prediction process. 

            elapsed_time = float(time.time() - start)
            reviews_per_second = i / elapsed_time if elapsed_time > 0 else 0
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                             + " #Correct:" + str(correct) + " #Tested:" + str(i+1) \
                             + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        """
        Returns a POSITIVE or NEGATIVE prediction for the given review.
        """
        # TODO: Run a forward pass through the network, like you did in the
        #       "train" function. That means use the given review to 
        #       update the input layer, then calculate values for the hidden layer,
        #       and finally calculate the output layer.
        #
        #       Note: The review passed into this function for prediction 
        #             might come from anywhere, so you should convert it 
        #             to lower case prior to using it.
        self.update_input_layer(review)
        layer_1 = self.layer_0.dot(self.weights_0_1)
        layer_2 = self.sigmoid(layer_1.dot(self.weights_1_2))
        
        # TODO: The output layer should now contain a prediction. 
        #       Return `POSITIVE` for predictions greater-than-or-equal-to `0.5`, 
        #       and `NEGATIVE` otherwise.
        if layer_2 >= 0.5: return 'POSITIVE'
        else: return 'NEGATIVE'
      
        
       
        
        
        

In [42]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.1)

In [43]:
mlp.test(reviews[-1000:],labels[-1000:])

Progress:13.9% Speed(reviews/sec):691.8 #Correct:70 #Tested:140 Testing Accuracy:50.0%

Progress:27.3% Speed(reviews/sec):701.9 #Correct:137 #Tested:274 Testing Accuracy:50.0%

Progress:40.3% Speed(reviews/sec):708.0 #Correct:202 #Tested:404 Testing Accuracy:50.0%

Progress:54.0% Speed(reviews/sec):704.5 #Correct:271 #Tested:541 Testing Accuracy:50.0%

Progress:67.7% Speed(reviews/sec):709.2 #Correct:339 #Tested:678 Testing Accuracy:50.0%

Progress:80.7% Speed(reviews/sec):707.3 #Correct:404 #Tested:808 Testing Accuracy:50.0%

Progress:93.8% Speed(reviews/sec):706.7 #Correct:470 #Tested:939 Testing Accuracy:50.0%

Progress:99.9% Speed(reviews/sec):709.0 #Correct:500 #Tested:1000 Testing Accuracy:50.0%

In [44]:
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):54.47 #Correct:2174 #Trained:2501 Training Accuracy:86.9%
Progress:20.8% Speed(reviews/sec):54.42 #Correct:4349 #Trained:5001 Training Accuracy:86.9%
Progress:31.2% Speed(reviews/sec):53.87 #Correct:6537 #Trained:7501 Training Accuracy:87.1%
Progress:41.6% Speed(reviews/sec):54.00 #Correct:8761 #Trained:10001 Training Accuracy:87.6%
Progress:52.0% Speed(reviews/sec):54.18 #Correct:10978 #Trained:12501 Training Accuracy:87.8%
Progress:62.5% Speed(reviews/sec):54.31 #Correct:13187 #Trained:15001 Training Accuracy:87.9%
Progress:72.9% Speed(reviews/sec):54.23 #Correct:15397 #Trained:17501 Training Accuracy:87.9%
Progress:83.3% Speed(reviews/sec):54.00 #Correct:17648 #Trained:20001 Training Accuracy:88.2%
Progress:93.7% Speed(reviews/sec):53.89 #Correct:19906 #Trained:22501 Training Accuracy:88.4%
Progress:99.9% Speed(reviews/sec):53.80 #Correct:21250 #Trained:24000 Training

In [45]:
mlp = SentimentNetwork(reviews[:-1000],labels[:-1000], learning_rate=0.01)
mlp.train(reviews[:-1000],labels[:-1000])

Progress:0.0% Speed(reviews/sec):0.0 #Correct:1 #Trained:1 Training Accuracy:100.%
Progress:10.4% Speed(reviews/sec):54.99 #Correct:2204 #Trained:2501 Training Accuracy:88.1%
Progress:20.8% Speed(reviews/sec):55.12 #Correct:4406 #Trained:5001 Training Accuracy:88.1%
Progress:31.2% Speed(reviews/sec):55.10 #Correct:6602 #Trained:7501 Training Accuracy:88.0%
Progress:41.6% Speed(reviews/sec):54.55 #Correct:8843 #Trained:10001 Training Accuracy:88.4%
Progress:52.0% Speed(reviews/sec):54.54 #Correct:11072 #Trained:12501 Training Accuracy:88.5%
Progress:62.5% Speed(reviews/sec):54.36 #Correct:13298 #Trained:15001 Training Accuracy:88.6%
Progress:72.9% Speed(reviews/sec):54.38 #Correct:15524 #Trained:17501 Training Accuracy:88.7%
Progress:83.3% Speed(reviews/sec):54.40 #Correct:17794 #Trained:20001 Training Accuracy:88.9%
Progress:93.7% Speed(reviews/sec):54.34 #Correct:20053 #Trained:22501 Training Accuracy:89.1%
Progress:99.9% Speed(reviews/sec):54.31 #Correct:21399 #Trained:24000 Training